# Basic information

## three fanspages
* imeifoods id='202586376996' 
* iLoveKCmilk id='180080052032125' 
* Dominos.tw id='115361891807579'

In [1]:
import pyes
import json
from IPython.core.display import display, HTML


host = '192.168.1.100:9200'
conn = pyes.es.ES(host)

fnames = ['imeifoods', 'iLoveKCmilk', 'Dominos.tw']
fids = ['202586376996', '180080052032125', '115361891807579']

tqs = [pyes.query.TermQuery(field="fid", value=fid) for fid in fids]
rs = [conn.search(query=tq, indices='facebook_nested', doc_types='fanpage') for tq in tqs]

print 'I choose three fanpages to analyze as following'
for r, name in zip(rs, fnames):
    display(HTML('<a href='+ json.dumps(r[0]['link'], indent=2) + ' target="_blank">' + name + '</a>'))

I choose three fanpages to analyze as following


## searching range (2015 whole year)

In [2]:
ESR = pyes.ESRange(field='created_time', from_value='2015-01-01T00:00:00+0800', to_value='2015-12-31T23:59:59+0800', 
                   include_lower=True, include_upper=False) # use whole data in 2015
rq = pyes.query.RangeQuery(qrange=ESR)
bqs = [pyes.query.BoolQuery().add_must(rq).add_must(tq) for tq in tqs]

## # of comments

In [3]:
res = [conn.search(query=bq, indices='facebook_nested', doc_types='comment') for bq in bqs]
print [len(r) for r in res]

[25556, 15444, 35136]


## # of posts

In [4]:
res = [conn.search(query=bq, indices='facebook_nested', doc_types='post') for bq in bqs]
print [len(r) for r in res]

[207, 105, 365]
